In [16]:
# https://ithelp.ithome.com.tw/articles/1018687
# 實作 MNIST Softmax 模型

# 目標
# 1實作 Softmax Regressions
# 2了解 tensorflow computation graph

import tensorflow as tf
#讀mnist
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


#操作符號變數來描述這一些交互操
#範例
#x 不是一個特定的數值．他是一個佔位子 (placeholder)
#預先保留的數值，在真正計算的時候才把數值輸入進去
#我們想要可以輸入任意數量的 MNIST 圖片
#每一張圖都會先轉化成 784 維的向量．
#用 2-D 的浮點數 tensor 來表現它
#None 意味著它第一個維度可以是任一長度的
x = tf.placeholder(tf.float32, [None, 784])


W = tf.Variable( tf.zeros([784,10]) )
b = tf.Variable( tf.zeros([10]) )

#softmax 回歸
#例如在郵件分類問題中，我們要把郵件分為垃圾郵件、個人郵件、工作郵件3類，目標值y是一個有3個取值的離散值。這是一個多分類問題
#目的是辨識10個不同的單個數字
y = tf.nn.softmax( tf.matmul(x,W) +b )

#模型訓練
#在機器學習中，通常是定義一個模型怎樣算是不好的
#這個定義稱作成本 (cost) 或是損失 (loss)，
#它代表我們的模型和預期的結果間的差距
#要最小化這些成本，成本或損失越低代表模型越好．
#常見成本函數稱作 cross-entropy
#y 是預測的機率分佈，
#而 y' 是真實的機率分佈 (one-hot 數字向量)
#cross-entropy 用來量測我們的預測和真實之間的差距

#放置正確的答案
#cross_entroy定義 -sum(yi' *log(yi))
y_ = tf.placeholder( tf.float32, [None,10])
cross_entropy = tf.reduce_mean( -tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))

# 使用梯度下降法 gradient descent algorithm 來最小化 cross_entropy
#learning rate 是 0.5
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

#執行
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#執行1000次
for i in range(1000):
    #從訓練數據中隨機抓取一批 100 筆數據
    #替換掉之前我們設定的位子 placeholder進行訓練
    #mnist真實的資料
    batch_xs, batch_ys = mnist.train.next_batch(100)
    
    sess.run(train_step, feed_dict={x: batch_xs, y_ :batch_ys})

#評估模型
#tf.argmax 可以讓我們找到在某一維的 tensor 中
#找到最大的數值的索引值 (index)
#tf.equal比對
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

#列出了一系列的布林值
#布林值轉化成浮點數然後取平均
#tf.cast(x輸入, dtype轉換目標類型, name=None名稱)類型轉換函數
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#印出測試資料執行的準確度
print( sess.run(accuracy,feed_dict = {
    x: mnist.test.images, y_:mnist.test.labels  
}))


#print(sess.run(W))
#print(W.shape)

sess.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0.9199
